# AI Toolkit by Ostris
## FLUX.1-schnell Training


In [2]:
!nvidia-smi

Mon Jan  5 10:26:27 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
!git clone https://github.com/ostris/ai-toolkit
!mkdir -p /content/dataset

Cloning into 'ai-toolkit'...
remote: Enumerating objects: 8415, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 8415 (delta 63), reused 32 (delta 32), pack-reused 8307 (from 3)
Receiving objects: 100% (8415/8415), 31.88 MiB | 33.52 MiB/s, done.
Resolving deltas: 100% (5893/5893), done.


Put your image dataset in the `/content/dataset` folder

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import subprocess

# Uninstall existing numpy and scipy to clear potential conflicts
print("Mevcut numpy kaldırılıyor...")
subprocess.run(['pip', 'uninstall', 'numpy', '-y'], check=False)
print("Mevcut scipy kaldırılıyor...")
subprocess.run(['pip', 'uninstall', 'scipy', '-y'], check=False)

# Install numpy 2.x which is required by many Colab pre-installed packages
# and is compatible with Python 3.12.
print("numpy==2.0.0 yükleniyor...")
subprocess.run(['pip', 'install', 'numpy==2.0.0'], check=True)
# Install scipy compatible with numpy 2.0.0. scipy 1.12.0 was picked by requirements.txt and should be compatible.
print("scipy==1.12.0 yükleniyor...")
subprocess.run(['pip', 'install', 'scipy==1.12.0'], check=True)

print("\nNumpy ve Scipy yeniden yüklendi. Lütfen ŞİMDİ ÇALIŞMA ZAMANINI YENİDEN BAŞLATIN (Çalışma Zamanı -> Çalışma zamanını yeniden başlat) ve ardından `git clone` hücresinden başlayarak notebook'u baştan çalıştırın.")

Mevcut numpy kaldırılıyor...
Mevcut scipy kaldırılıyor...
numpy==2.0.0 yükleniyor...
scipy==1.12.0 yükleniyor...

Numpy ve Scipy yeniden yüklendi. Lütfen ŞİMDİ ÇALIŞMA ZAMANINI YENİDEN BAŞLATIN (Çalışma Zamanı -> Çalışma zamanını yeniden başlat) ve ardından `git clone` hücresinden başlayarak notebook'u baştan çalıştırın.


In [1]:
!cd ai-toolkit && git submodule update --init --recursive && pip install -r requirements.txt


  Cloning https://github.com/huggingface/diffusers (to revision f6b6a7181eb44f0120b29cd897c129275f366c2a) to /tmp/pip-req-build-rtmtvuk6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-rtmtvuk6
  Running command git rev-parse -q --verify 'sha^f6b6a7181eb44f0120b29cd897c129275f366c2a'
  Running command git fetch -q https://github.com/huggingface/diffusers f6b6a7181eb44f0120b29cd897c129275f366c2a
  Running command git checkout -q f6b6a7181eb44f0120b29cd897c129275f366c2a
  Resolved https://github.com/huggingface/diffusers to commit f6b6a7181eb44f0120b29cd897c129275f366c2a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Model License
Training currently only works with FLUX.1-dev. Which means anything you train will inherit the non-commercial license. It is also a gated model, so you need to accept the license on HF before using it. Otherwise, this will fail. Here are the required steps to setup a license.

Sign into HF and accept the model access here [black-forest-labs/FLUX.1-dev](https://huggingface.co/black-forest-labs/FLUX.1-dev)

[Get a READ key from huggingface](https://huggingface.co/settings/tokens/new?) and place it in the next cell after running it.

In [2]:
import getpass
import os

# Prompt for the token
hf_token = getpass.getpass('Enter your HF access token and press enter: ')

# Set the environment variable
os.environ['HF_TOKEN'] = hf_token

print("HF_TOKEN environment variable has been set.")

Enter your HF access token and press enter: ··········
HF_TOKEN environment variable has been set.


In [3]:
import os
import sys
sys.path.append('/content/ai-toolkit')
from toolkit.job import run_job
from collections import OrderedDict
from PIL import Image
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

## Setup

This is your config. It is documented pretty well. Normally you would do this as a yaml file, but for colab, this will work. This will run as is without modification, but feel free to edit as you want.

In [4]:
from collections import OrderedDict
import os

# Set environment variable to help with CUDA memory fragmentation
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

job_to_run = OrderedDict([
    ('job', 'extension'),
    ('config', OrderedDict([
        ('name', 'loomee_flux_lora_v1'),
        ('process', [
            OrderedDict([
                ('type', 'sd_trainer'),
                ('training_folder', '/content/output'),
                ('device', 'cuda:0'),

                # BURAYI DEĞİŞTİRDİK: Trigger word tanımlandı
                ('trigger_word', 'LOOMEE_LDVN1'),
                ('low_vram', True), # ADDED FOR LOW VRAM USAGE

                ('network', OrderedDict([
                    ('type', 'lora'),
                    ('linear', 16),
                    ('linear_alpha', 16)
                ])),
                ('save', OrderedDict([
                    ('dtype', 'float16'),
                    ('save_every', 250),
                    ('max_step_saves_to_keep', 4)
                ])),
                ('datasets', [
                    OrderedDict([
                        ('folder_path', '/content/drive/MyDrive/loomee_training'),
                        ('caption_ext', 'txt'),
                        ('caption_dropout_rate', 0.05),
                        ('shuffle_tokens', False),
                        ('cache_latents_to_disk', True),
                        # BURAYI DEĞİŞTİRDİK: Bellek kullanımı için çözünürlük düşürüldü
                        ('resolution', 512)
                    ])
                ]),
                ('train', OrderedDict([
                    ('batch_size', 1),
                    ('steps', 2000),
                    ('gradient_accumulation_steps', 1),
                    ('train_unet', True),
                    ('train_text_encoder', False),
                    ('gradient_checkpointing', True),
                    ('noise_scheduler', 'flowmatch'),
                    ('optimizer', 'adamw8bit'),
                    ('lr', 1e-4),
                    ('ema_config', OrderedDict([
                        ('use_ema', True),
                        ('ema_decay', 0.99)
                    ])),
                    ('dtype', 'bf16')
                ])),
                ('model', OrderedDict([
                    ('name_or_path', 'black-forest-labs/FLUX.1-schnell'),
                    ('assistant_lora_path', 'ostris/FLUX.1-schnell-training-adapter'),
                    ('is_flux', True),
                    ('quantize', True)
                ])),
                ('sample', OrderedDict([
                    ('sampler', 'flowmatch'),
                    ('sample_every', 250),
                    ('width', 1024),
                    ('height', 1024),
                    ('prompts', [
                        # BURAYI DEĞİŞTİRDİK: Loomee'ye özel promptlar eklendi
                        '[trigger] smart book light, sitting on a wooden nightstand, soft warm glow, high-end product photography',
                        'a person reading a book in a dark room, [trigger] clipped to the book, cinematic lighting, cozy atmosphere',
                        'close up of [trigger], focusing on the sleek design and high-quality texture, white background, studio lighting',
                        'a modern home office desk, [trigger] is clipped to a notebook, moody ambient lighting, professional tech aesthetic',
                        '[trigger] holding a small sign that says \'LOOMEE\', minimalist background, sharp focus',
                        'a bulldog in a leather jacket sitting in a library, reading a book with the help of [trigger], cinematic style'
                    ]),
                    ('neg', ''),
                    ('seed', 42),
                    ('walk_seed', True),
                    ('guidance_scale', 1),
                    ('sample_steps', 4)
                ]))
            ])
        ])
    ])),
    ('meta', OrderedDict([
        ('name', '[name]'),
        ('version', '1.0')
    ]))
])

## Run it

Below does all the magic. Check your folders to the left. Items will be in output/LoRA/your_name_v1 In the samples folder, there are preiodic sampled. This doesnt work great with colab. They will be in /content/output

In [5]:
run_job(job_to_run)


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
/usr/local/lib/python3.12/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


{
    "type": "sd_trainer",
    "training_folder": "/content/output",
    "device": "cuda:0",
    "trigger_word": "LOOMEE_LDVN1",
    "low_vram": true,
    "network": {
        "type": "lora",
        "linear": 16,
        "linear_alpha": 16
    },
    "save": {
        "dtype": "float16",
        "save_every": 250,
        "max_step_saves_to_keep": 4
    },
    "datasets": [
        {
            "folder_path": "/content/drive/MyDrive/loomee_training",
            "caption_ext": "txt",
            "caption_dropout_rate": 0.05,
            "shuffle_tokens": false,
            "cache_latents_to_disk": true,
            "resolution": 512
        }
    ],
    "train": {
        "batch_size": 1,
        "steps": 2000,
        "gradient_accumulation_steps": 1,
        "train_unet": true,
        "train_text_encoder": false,
        "gradient_checkpointing": true,
        "noise_scheduler": "flowmatch",
        "optimizer": "adamw8bit",
        "lr": 0.0001,
        "ema_config": {
         

config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

(…)ion_pytorch_model.safetensors.index.json:   0%|          | 0.00/121k [00:00<?, ?B/s]

transformer/diffusion_pytorch_model-0000(…):   0%|          | 0.00/9.96G [00:00<?, ?B/s]

...


Generating Images: 100%|██████████| 6/6 [00:29<00:00,  4.90s/it]
                                                                

Loading assistant lora


loomee_flux_lora_v1: 100%|█████████▉| 1999/2000 [52:31<00:01,  1.58s/it, lr: 1.0e-04 loss: 4.011e-01]


Unloading assistant lora


Loading assistant lora

Saved checkpoint to /content/output/loomee_flux_lora_v1/loomee_flux_lora_v1.safetensors
Saved optimizer to /content/output/loomee_flux_lora_v1/optimizer.pt


## Done

Check your ourput dir and get your slider
